In [ ]:
import asyncio
import os
import shutil
import nest_asyncio
from tqdm import tqdm
from threading import Thread
from google.colab import drive

if not os.path.exists('/content/drive/MyDrive'):
    print('Гугл диск...')
    drive.mount('/content/drive')

!pip install -q yt-dlp

if not os.path.exists('/content/downloads'):
    !mkdir /content/downloads
if not os.path.exists('/content/drive/MyDrive/downloads'):
    !mkdir /content/drive/MyDrive/downloads

#@title Загрузчик видео с сайтов
URL_LINK_1 = "" #@param {type:"string"}
URL_LINK_2 = "" #@param {type:"string"}
URL_LINK_3 = "" #@param {type:"string"}
URL_LINK_4 = "" #@param {type:"string"}
URL_LINK_5 = "" #@param {type:"string"}
URL_LINK_6 = "" #@param {type:"string"}
URL_LINK_7 = "" #@param {type:"string"}
URL_LINK_8 = "" #@param {type:"string"}
URL_LINK_9 = "" #@param {type:"string"}
URL_LINK_10 = "" #@param {type:"string"}

#@markdown  <br> Тип выполнения процесса (asyncio - оптимально асинхронная загрузка; threading - в несколько потоков)
TYPE_PROCESS = "asyncio" #@param ["asyncio", "threading"]

#@markdown  <br> Очистить папку content/downloads
WIPE_INTERNAL_DOWNLOADS = False #@param {type:"boolean"}

if WIPE_INTERNAL_DOWNLOADS:
    !rm -rf /content/downloads

# Список URL-ссылок
URL_LINKS = []

# Добавление URL_LINK_* в список URL_LINKS
for i in range(1, 11):
    url_var_name = f"URL_LINK_{i}"
    url = globals().get(url_var_name)
    if url:
        URL_LINKS.append(url)


if TYPE_PROCESS == "asyncio":

    # Функция для выполнения команд в отдельном процессе
    async def execute_command(command):
        process = await asyncio.create_subprocess_shell(command, stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE)
        _, _ = await process.communicate()

    # Создание директорий для загрузок
    os.makedirs("/content/downloads", exist_ok=True)
    os.makedirs("/content/drive/MyDrive/downloads", exist_ok=True)

    # Создание и запуск асинхронных задач
    async def main():
        tasks = []
        for url in URL_LINKS:
            command = f'yt-dlp -P /content/downloads -- {url}'
            task = asyncio.create_task(execute_command(command))
            tasks.append(task)

        # Отображение прогресса выполнения задач
        with tqdm(total=len(tasks), desc="Загрузка") as pbar:
            for task in asyncio.as_completed(tasks):
                await task
                pbar.update(1)

        # Копирование файлов в директорию на Google Диске
        shutil.copytree("/content/downloads", "/content/drive/MyDrive/downloads", dirs_exist_ok=True)


    # Запуск основной программы с использованием nest_asyncio
    nest_asyncio.apply()
    await main()

if TYPE_PROCESS == "threading":

    # Функция для выполнения команд в отдельном потоке
    def execute_command(command):
        !{command}

    # Создание и запуск потоков для выполнения команд
    threads = []
    for url in URL_LINKS:
        command = f'yt-dlp -P /content/downloads -- {url}'
        thread = Thread(target=execute_command, args=(command,))
        thread.start()
        threads.append(thread)

    # Ожидание завершения всех потоков
    for thread in threads:
        thread.join()

    # Копирование файлов в директорию на Google Диске
    !cp -rf /content/downloads/* /content/drive/MyDrive/downloads


print('Завершено!')
